# Analysing the redundancies of User Stories - Just the use Stories

In [2]:
import locale
import os, json, re
from itertools import combinations

import pandas as pd
import tiktoken
from openpyxl import Workbook
from openpyxl.styles import Alignment, Font
from openpyxl.utils import get_column_letter as utils_get_column_letter

from prompt_structure.helper_prompt_composition import PromptHelperBuilder
from prompt_structure.prompt_builder import PromptBuilder

from support_functions.excel_helper import save_to_excel, formatter_ignored_items
from support_functions.load_data import load_datasets_with_annotations as loading
from support_functions.request_handler import process_user_stories, process_user_stories_parallel
from support_functions.data_transformations import convert_annotation_dataset
from support_functions.time_recorder import TimeRecorder
from support_functions.json_validator import validation, chat_gpt_text_output

In [3]:
locale.setlocale(locale.LC_ALL, 'de_DE') 
MODEL_VERSION_NAME = os.getenv('MODEL_VERSION')
THREADING = bool(int(os.getenv('THREADING')))
print(f"Threading is used: {THREADING}")
print(f"Model Version: {MODEL_VERSION_NAME}")
print(f"Threads: {os.getenv("THREAD_MULTIPLICATOR")}")
print(f"Limit: {os.getenv("LIMIT")}")

Threading is used: False
Model Version: gpt-4o-mini-2024-07-18
Threads: 0.8
Limit: 1


In [4]:
time_recorders: dict = {}
time_recorder: TimeRecorder = None

### Load all User Stories from the different packages

We use here just the pure text representation of a User Story seperated:
- Main Part
- Benefit

We do not need:
- annotations

Ignored Files as they differer from the common structure [Ref](https://github.com/ace-design/nlp-stories/tree/main?tab=readme-ov-file#note):
- g02-federal-funding
- g13-planningpoker
- g17-cask
- g27-culrepo

Ignored Files as they were used as trainings data
- g05
- g12

Data sets to analyse:
 - 'g19', 'g22'

Used Data:
- "nlp" --> "nlp_outputs" --> "individual_backlog" --> "nlp_outputs_original" --> "pos_baseline"

Highlighting words with #

Datensätze mit mehr als 70 Einträgen müssen single threaded laufen

In [5]:
not_processed_datasets: dict[str, list] = loading()
print(not_processed_datasets["g02"][0])

{'PID': '#G02#', 'Text': '#G02# As a Data user, I want to have the 12-19-2017 deletions processed.', 'Persona': ['Data user'], 'Action': {'Primary Action': ['have'], 'Secondary Action': ['processed']}, 'Entity': {'Primary Entity': ['12-19-2017 deletions'], 'Secondary Entity': ['']}, 'Benefit': '', 'Triggers': [['Data user', 'have']], 'Targets': [['processed', '12-19-2017 deletions'], ['have', '12-19-2017 deletions']], 'Contains': [], 'Persona POS': {'Persona POS tag': [['NOUN', 'NOUN']], 'Persona POS text': [['Data', 'user']]}, 'Action POS': {'Primary Action POS': {'Primary Action POS tag': [['VERB']], 'Primary Action POS text': [['have']]}, 'Secondary Action POS': {'Secondary Action POS tag': [['VERB']], 'Secondary Action POS text': [['processed']]}}, 'Entity POS': {'Primary Entity POS': {'Primary Entity POS tag': [['NUM', 'NOUN']], 'Primary Entity POS text': [['12-19-2017', 'deletions']]}, 'Secondary Entity POS': {'Secondary Entity POS tag': [[]], 'Secondary Entity POS text': [[]]}}}

### Converting Data to used suited format

In [6]:
datasets, ignored_items = convert_annotation_dataset(not_processed_datasets)

In [7]:
datasets.keys()

dict_keys(['g02', 'g03', 'g04', 'g05', 'g08', 'g10', 'g11', 'g12', 'g13', 'g14', 'g16', 'g17', 'g18', 'g19', 'g21', 'g22', 'g23', 'g24', 'g25', 'g26', 'g27', 'g28'])

In [8]:
datasets["g22"]

[{'PID': '#G22#',
  'USID': '1225',
  'Text': 'As a PI, I want to properly record all metadata, so that I can ensure proper running of the project in case of staff changes.',
  'Main Part': 'As a PI, I want to properly record all metadata',
  'Benefit': 'I can ensure proper running of the project in case of staff changes',
  'Triggers': {'Main Part': [['PI', 'properly record']], 'Benefit': []},
  'Targets': {'Main Part': [['properly record', 'all metadata']],
   'Benefit': [['ensure', 'proper running'], ['changes', 'staff']]},
  'Contains': {'Main Part': [], 'Benefit': []}},
 {'PID': '#G22#',
  'USID': '1226',
  'Text': 'As a researcher, I want to indicate where my data will be stored during the project and where after the project, so that it remains accessible during all stages of the data lifecycle.',
  'Main Part': 'As a researcher, I want to indicate where my data will be stored during the project and where after the project',
  'Benefit': 'it remains accessible during all stages o

In [9]:
ignored_items["#G02#"][0]

'PID: #G02#; Text: As a UI designer, I want to redesign the Resources page, so that it matches the new Broker design styles.; Label Type: Contain'

In [10]:
df_ignored_items: pd.DataFrame = None
if True:            
    cols: list[str] =  ["PID", "Text", "Label Type"]
    df_ignored_items = pd.DataFrame(columns=cols)
    current_ignored_data: dict = {}
    curremt_pid: int = 0
    current_text: str = "" 
    for key, set in ignored_items.items():
        for item in set:
            parts = item.split('; ')
            for part in parts:
                key, value = part.split(': ', 1)
                current_ignored_data[key] = value
            df_ignored_items = pd.concat([df_ignored_items, pd.DataFrame([current_ignored_data])], ignore_index = True)               
            df_ignored_items.reset_index()
    save_to_excel(df_ignored_items, formatter_ignored_items, "Incomplete US labelling", os.getenv("OUTPUT_EXCEL_NAME_JUST_TEXTUAL"))
print(f"Count of ignored Elements: {df_ignored_items.shape[0]}")
df_ignored_items.head()

Count of ignored Elements: 195


,PID,Text,Label Type
0,#G02#,"As a UI designer, I want to redesign the Resou...",Contain
1,#G02#,"As a UI designer, I want to track the issues t...",Target
2,#G02#,"As a UI designer, I want to track the issues t...",Target
3,#G02#,"As a UI designer, I want to track the issues t...",Target
4,#G02#,"As an agency user, I want to submit my data el...",Contain


### Datatransformation to panda dataframes and numpy arrarys. 

In [11]:
def transform_2_dataframe(data: list):
    def remove_pattern(text: str, pattern: str) -> str:
        if re.match(pattern, text):
            text = re.sub(pattern, '', text)
            text.strip()
        return text
    pattern = r'#G\d{2}#'
    
    rows = []
    for entry in data:
        entry["Main Part"] = remove_pattern(entry["Main Part"], pattern)
        row = {
            "PID": entry["PID"],
            "USID": entry["USID"],
            "Text": entry["Text"],
            "Main Part": entry["Main Part"],
            "Benefit": entry["Benefit"],
            "Triggers": entry["Triggers"],
            "Targets": entry["Targets"],
            "Contains": entry["Contains"],
        }
        rows.append(row)
    return pd.DataFrame(rows)

In [12]:
# n(n-1) / 2
def transform_pairwise(df: pd.DataFrame):
    rows = []
    for i, j in combinations(range(df.shape[0]), 2):
        row = {
            "First USID": df.iloc[i, 1],
            "First Text": df.iloc[i, 2],
            "First Main Part": df.iloc[i, 3],
            "First Benefit": df.iloc[i, 4],
            "Second USID": df.iloc[j, 1],
            "Second Text": df.iloc[j, 2],
            "Second Main Part": df.iloc[j, 3],
            "Second Benefit": df.iloc[j, 4],
        }
        rows.append(row)
    return pd.DataFrame(rows)

In [13]:
def convert_dataframe_to_json(pair: pd.Series) -> tuple[dict, dict]:    
    us_one: dict = {
        "USID": pair["First USID"],
        "Text": pair["First Text"],
        "Main Part": pair["First Main Part"],
        "Benefit": pair["First Benefit"],
    }
    
    us_two: dict = {
        "USID": pair["Second USID"],
        "Text": pair["Second Text"],
        "Main Part": pair["Second Main Part"],
        "Benefit": pair["Second Benefit"],
    }

    json_one: dict = json.loads(json.dumps(us_one))
    json_two: dict = json.loads(json.dumps(us_two))
    
    return json_one, json_two

In [14]:
def template_request_two_user_stories(
    current_message: list[dict],
    idx: int,
    pairs: pd.DataFrame,
) -> None:
    row = pairs.iloc[idx]
    json_us_one: dict = None
    json_us_two: dict = None
    json_us_one, json_us_two = convert_dataframe_to_json(row)
    request: dict = PromptHelperBuilder.get_instance().parsing_to_pair_requests(json_us_one, json_us_two)
    current_message.append(request)

In [15]:
def sort_threaded_results(to_sort: dict) -> None:
    new_list: list = None
    for _ in to_sort.values():
        new_list = sorted(_, key=lambda x: (int(x['relatedStories'][0]), int(x['relatedStories'][1])))
        _.clear()
        _ += new_list

In [16]:
def json_validator(json_data: dict) -> tuple[bool, str]:
    return validation(json_data, chat_gpt_text_output)

### Prepaire Prompting

In [17]:
# Init PromptBuilder
builder: PromptBuilder = PromptBuilder.get_instance()

# Collection of messages
message: list[dict] = []

In [18]:
# Actor Role
message.append(builder.get_actor_role())
message.append(builder.get_system_simulation_actor_role())

In [19]:
# User Story Definition
message.append(builder.get_user_story_definition())
message.append(builder.get_system_simulation_user_story_definition_only_text())

In [20]:
# Task and fucos definition

message.append(builder.get_task_focus_definition_only_text())
message.append(builder.get_system_simulation_task_focus_definition())

In [21]:
# Redundancy Definition for User Story pair
message.append(builder.get_redundancy_strict_definition_text())
message.append(builder.get_system_simulation_redundancy())

In [22]:
builder.get_redundancy_strict_definition_text()

{'role': 'user',
 'content': "Two user stories are redundant in the main part if one value of 'Targets'.'Main Part' of the first story also occurs in 'Targets'.'Main Part' of the second story. Hereby, we are just looking for exact duplicate words. I.e., they spelled identically. If we consider user story one which contains ['use', 'dataset'] in 'Targets'.'Main Part' and user story two which contains ['use', 'dataset'] in 'Targets'.'Main Part', they are redundant in the main part. If we consider user story one which contains ['use', 'dataset'] in 'Targets'.'Main Part' and user story two which contains ['use', 'data'] in 'Targets'.'Main Part', they are not redundant in the main part. Two user stories are redundant in the benefit if one value of 'Targets'.Benefit' of the first story also occurs in 'Targets'.Benefit' of the second story. Hereby, we are just looking for exact duplicate words. I.e., they are spelled identically. If we consider user story one which contains ['use', 'dataset']

In [23]:
# Defining the JSON output
message.append(builder.get_json_format_defintion_text())
message.append(builder.get_system_simulation_json_format_defintion())

In [ ]:
# Providing Examples
message.append(builder.get_input_output_examples(["USID", "Triggers", "Targets", "Contains"]))
message.append(builder.get_system_simulation_example_consideration())

In [23]:
if True:
    message_text: str = ""
    for key in message:
        message_text += key["content"]
        print(key["content"])
    enc = tiktoken.get_encoding('cl100k_base')
    token_size = enc.encode(message_text)
    # print("-" * 3 + "Token" + "-" * 3)
    # print(token_size)
    print("-" * 3 + "Sum of Tokens" + "-" * 3)
    # The output is not correct as the result from the online pages differs https://platform.openai.com/tokenizer
    print("The total token sum is: " + locale.format_string("%d", len(token_size), grouping=True))
    

Act as a Requirements Engineer focused on identifying redundancies. Please review pairs of two User Stories and pinpoint any unnecessary duplications that obscure clarity or add no distinct value.
As a requirements engineer in agile development, it is my responsibility to review user stories for redundancies. My goal is to identify and report any overlapping or duplicate requirements. By carefully analysing the user stories in depth, I ensure that each requirement is necessary and contributes uniquely, increasing the coherence of the product.
A User Story is a semi-structured sentence containing the following information:
(1) the persona involved in the story,
(2) the main part containing the actions that the persona will perform on the system and the entities involved in the actions, and optionally
(3) a benefit that the persona will receive after having completed these actions. The benefit may also include actions and entities.
Classically, a User Story is expressed in the following 

---Sum of Tokens---
The total token sum is: 5.364


## Example generation for the paper - the listed US are not contained in the trainings data (manuel removed before this analyse from the input_examples.josn and output_examples.json)

In [24]:
ID_G05: str = "g05"
data_g05: list = datasets[ID_G05]
df_g05: pd.DataFrame = transform_2_dataframe(data_g05)
df_g05_pairs = transform_pairwise(df_g05)

target_usids = [[408, 409]]

def is_in_target_usids(usid_one: int, usid_two: int):
    for element in target_usids:
        if (usid_one == element[0] and usid_two == element[1]):
            return True
    return False

# Apply the function to each row
df_g05_pairs_adjusted = df_g05_pairs[
    df_g05_pairs.apply(lambda row: is_in_target_usids(int(row['First USID']), int(row['Second USID'])), axis=1)
]

df_g05_pairs_adjusted

,First USID,First Text,First Main Part,First Benefit,Second USID,Second Text,Second Main Part,Second Benefit
1225,408,"As an API User, I want to be able to normalise...","As an API User, I want to be able to normalise...",I work with datasets in reference to their con...,409,"As an API User, I want to be able to normalise...","As an API User, I want to be able to normalise...",I work with datasets in reference to their con...


## Data processing for: G19

In [25]:
ID_G19: str = "g19"
data_g19: list = datasets[ID_G19]
df_g19: pd.DataFrame = transform_2_dataframe(data_g19)
df_g19_pairs = transform_pairwise(df_g19)

In [26]:
datasets[ID_G19][0]

{'PID': '#G19#',
 'USID': '1018',
 'Text': 'As an OlderPerson, I want to know exactly what ALFRED does with my personal data, and share it only on my specific permission.',
 'Main Part': ' As an OlderPerson, I want to know exactly what ALFRED does with my personal data, and share it only on my specific permission.',
 'Benefit': '',
 'Triggers': {'Main Part': [['OlderPerson', 'know exactly']], 'Benefit': []},
 'Targets': {'Main Part': [['know exactly', 'what']],
  'Benefit': [['share', 'personal data']]},
 'Contains': {'Main Part': [], 'Benefit': [['ALFRED', 'personal data']]}}

In [27]:
display(df_g19.head(5))

,PID,USID,Text,Main Part,Benefit,Triggers,Targets,Contains
0,#G19#,1018,"As an OlderPerson, I want to know exactly what...","As an OlderPerson, I want to know exactly wha...",,"{'Main Part': [['OlderPerson', 'know exactly']...","{'Main Part': [['know exactly', 'what']], 'Ben...","{'Main Part': [], 'Benefit': [['ALFRED', 'pers..."
1,#G19#,1019,"As an OlderPerson, I want to receive informati...","As an OlderPerson, I want to receive informat...",,"{'Main Part': [['OlderPerson', 'receive']], 'B...","{'Main Part': [['receive', 'information']], 'B...","{'Main Part': [['ALFRED', 'information']], 'Be..."
2,#G19#,1020,"As an OlderPerson, I want to use only well-vis...","As an OlderPerson, I want to use only well-vi...",,"{'Main Part': [['OlderPerson', 'use only']], '...","{'Main Part': [['use only', 'well-visible butt...","{'Main Part': [], 'Benefit': []}"
3,#G19#,1021,"As an OlderPerson, I want to use ALFRED as muc...","As an OlderPerson, I want to use ALFRED as mu...",,"{'Main Part': [['OlderPerson', 'use']], 'Benef...","{'Main Part': [['use', 'ALFRED']], 'Benefit': []}","{'Main Part': [['ALFRED', 'speech interaction'..."
4,#G19#,1022,"As an OlderPerson, I want to always keep contr...","As an OlderPerson, I want to always keep cont...",,"{'Main Part': [['OlderPerson', 'always keep co...","{'Main Part': [['always keep control', 'activi...","{'Main Part': [['ALFRED', 'activities']], 'Ben..."


In [28]:
display(df_g19_pairs.head())

,First USID,First Text,First Main Part,First Benefit,Second USID,Second Text,Second Main Part,Second Benefit
0,1018,"As an OlderPerson, I want to know exactly what...","As an OlderPerson, I want to know exactly wha...",,1019,"As an OlderPerson, I want to receive informati...","As an OlderPerson, I want to receive informat...",
1,1018,"As an OlderPerson, I want to know exactly what...","As an OlderPerson, I want to know exactly wha...",,1020,"As an OlderPerson, I want to use only well-vis...","As an OlderPerson, I want to use only well-vi...",
2,1018,"As an OlderPerson, I want to know exactly what...","As an OlderPerson, I want to know exactly wha...",,1021,"As an OlderPerson, I want to use ALFRED as muc...","As an OlderPerson, I want to use ALFRED as mu...",
3,1018,"As an OlderPerson, I want to know exactly what...","As an OlderPerson, I want to know exactly wha...",,1022,"As an OlderPerson, I want to always keep contr...","As an OlderPerson, I want to always keep cont...",
4,1018,"As an OlderPerson, I want to know exactly what...","As an OlderPerson, I want to know exactly wha...",,1023,"As an OlderPerson, I want to be able to manage...","As an OlderPerson, I want to be able to manag...",


In [29]:
len(df_g19_pairs)

9316

### Process and Store the User Stories persistently

In [30]:
time_recorder = TimeRecorder()
time_recorders[ID_G19] = time_recorder

if THREADING:
    process_user_stories_parallel(index_usid1=0, index_usid2=4, message=message, pairs=df_g19_pairs, key=ID_G19, model_version_name=MODEL_VERSION_NAME, template_request_two_user_stories=template_request_two_user_stories,
                                  sort_threaded_results=sort_threaded_results, json_schema=chat_gpt_text_output, redundancy_prefix="text", time_recorder=time_recorder)
else:
    process_user_stories(index_usid1=0, index_usid2=0, message=message, pairs=df_g19_pairs, key=ID_G19, model_version_name=MODEL_VERSION_NAME, template_request_two_user_stories=template_request_two_user_stories, 
                         json_validation=json_validator, redundancy_prefix="text", time_recorder=time_recorder)

In [31]:
# target_usids = []
# def is_in_target_usids(usid_one: int, usid_two: int):
#     for element in target_usids:
#         if (usid_one == element[0] and usid_two == element[1]):
#             return True
#     return False

# # Apply the function to each row
# df_g19_pairs_adjusted = df_g19_pairs[
#     df_g19_pairs.apply(lambda row: is_in_target_usids(int(row['First USID']), int(row['Second USID'])), axis=1)
# ]

# print(len(df_g19_pairs_adjusted))
# df_g19_pairs_adjusted.head(5)

In [32]:
# process_user_stories(index_usid1=0, index_usid2=4, message=message, pairs=df_g19_pairs_adjusted, key=ID_G19, model_version_name=MODEL_VERSION_NAME, template_request_two_user_stories=template_request_two_user_stories, 
#                          json_validation=json_validator, redundancy_prefix="annoations", time_recorder=time_recorder)

In [33]:
# process_user_stories(index_usid1=1, index_usid2=9, message=message, pairs=df_g05_pairs_adjusted, key=ID_G05, model_version_name=MODEL_VERSION_NAME, template_request_two_user_stories=template_request_two_user_stories, 
#                          json_validation=json_validator, redundancy_prefix="annoations", time_recorder=time_recorder)

## Data processing for: G22

In [31]:
ID_G22: str = "g22"
data_g22: list = datasets[ID_G22]
df_g22: pd.DataFrame = transform_2_dataframe(data_g22)
df_g22_pairs = transform_pairwise(df_g22)

In [32]:
datasets[ID_G22][0]

{'PID': '#G22#',
 'USID': '1225',
 'Text': 'As a PI, I want to properly record all metadata, so that I can ensure proper running of the project in case of staff changes.',
 'Main Part': 'As a PI, I want to properly record all metadata',
 'Benefit': 'I can ensure proper running of the project in case of staff changes',
 'Triggers': {'Main Part': [['PI', 'properly record']], 'Benefit': []},
 'Targets': {'Main Part': [['properly record', 'all metadata']],
  'Benefit': [['ensure', 'proper running'], ['changes', 'staff']]},
 'Contains': {'Main Part': [], 'Benefit': []}}

In [33]:
display(df_g22.head(5))

,PID,USID,Text,Main Part,Benefit,Triggers,Targets,Contains
0,#G22#,1225,"As a PI, I want to properly record all metadat...","As a PI, I want to properly record all metadata",I can ensure proper running of the project in ...,"{'Main Part': [['PI', 'properly record']], 'Be...","{'Main Part': [['properly record', 'all metada...","{'Main Part': [], 'Benefit': []}"
1,#G22#,1226,"As a researcher, I want to indicate where my d...","As a researcher, I want to indicate where my d...",it remains accessible during all stages of the...,"{'Main Part': [['researcher', 'indicate']], 'B...","{'Main Part': [['indicate', 'where'], ['indica...","{'Main Part': [], 'Benefit': [['where', 'data'..."
2,#G22#,1227,"As a researcher, I want to import metadata tha...","As a researcher, I want to import metadata tha...",I do not have to capture it again in a DMP,"{'Main Part': [['researcher', 'import']], 'Ben...","{'Main Part': [['import', 'metadata']], 'Benef...","{'Main Part': [], 'Benefit': []}"
3,#G22#,1228,"As a data manager, I want to know how the data...","As a data manager, I want to know how the data...",I can develop more detailed usage and support ...,"{'Main Part': [['data manager', 'know']], 'Ben...","{'Main Part': [['know', 'how']], 'Benefit': [[...","{'Main Part': [], 'Benefit': []}"
4,#G22#,1229,"As a repository owner, I want to be able to ch...","As a repository owner, I want to be able to ch...",I can calculate necessary submission fees and ...,"{'Main Part': [['repository owner', 'check']],...","{'Main Part': [['check', 'planned provenance']...","{'Main Part': [['data', 'planned provenance']]..."


In [34]:
display(df_g22_pairs.head(5))

,First USID,First Text,First Main Part,First Benefit,Second USID,Second Text,Second Main Part,Second Benefit
0,1225,"As a PI, I want to properly record all metadat...","As a PI, I want to properly record all metadata",I can ensure proper running of the project in ...,1226,"As a researcher, I want to indicate where my d...","As a researcher, I want to indicate where my d...",it remains accessible during all stages of the...
1,1225,"As a PI, I want to properly record all metadat...","As a PI, I want to properly record all metadata",I can ensure proper running of the project in ...,1227,"As a researcher, I want to import metadata tha...","As a researcher, I want to import metadata tha...",I do not have to capture it again in a DMP
2,1225,"As a PI, I want to properly record all metadat...","As a PI, I want to properly record all metadata",I can ensure proper running of the project in ...,1228,"As a data manager, I want to know how the data...","As a data manager, I want to know how the data...",I can develop more detailed usage and support ...
3,1225,"As a PI, I want to properly record all metadat...","As a PI, I want to properly record all metadata",I can ensure proper running of the project in ...,1229,"As a repository owner, I want to be able to ch...","As a repository owner, I want to be able to ch...",I can calculate necessary submission fees and ...
4,1225,"As a PI, I want to properly record all metadat...","As a PI, I want to properly record all metadata",I can ensure proper running of the project in ...,1230,"As an IT manager, I want to know about IT reso...","As an IT manager, I want to know about IT reso...",I can enable resource acquisition planning


In [35]:
len(df_g22_pairs)

3403

### Process and Store the User Stories persistently

In [36]:
time_recorder = TimeRecorder()
time_recorders[ID_G22] = time_recorder

if THREADING:
    process_user_stories_parallel(index_usid1=0, index_usid2=4, message=message, pairs=df_g22_pairs, key=ID_G22, model_version_name=MODEL_VERSION_NAME, template_request_two_user_stories=template_request_two_user_stories,
                                  sort_threaded_results=sort_threaded_results, json_schema=chat_gpt_text_output, redundancy_prefix="text", time_recorder=time_recorder)
else:
    process_user_stories(index_usid1=0, index_usid2=4, message=message, pairs=df_g22_pairs, key=ID_G22, model_version_name=MODEL_VERSION_NAME, template_request_two_user_stories=template_request_two_user_stories, 
                         json_validation=json_validator, redundancy_prefix="text", time_recorder=time_recorder)

### Analyse broken ones

In [37]:
target_usids = []

def is_in_target_usids(usid_one: int, usid_two: int):
    for element in target_usids:
        if (usid_one == element[0] and usid_two == element[1]):
            return True
    return False

# Apply the function to each row
df_g22_pairs_adjusted = df_g22_pairs[
    df_g22_pairs.apply(lambda row: is_in_target_usids(int(row['First USID']), int(row['Second USID'])), axis=1)
]

print(len(df_g22_pairs_adjusted))
df_g22_pairs_adjusted.head(5)

0


,First USID,First Text,First Main Part,First Benefit,Second USID,Second Text,Second Main Part,Second Benefit


In [38]:
# process_user_stories(index_usid1=1, index_usid2=9, message=message, pairs=df_g22_pairs_adjusted, key=ID_G22, model_version_name=MODEL_VERSION_NAME, template_request_two_user_stories=template_request_two_user_stories, 
#                          json_validation=json_validator, redundancy_prefix="annoations", time_recorder=time_recorder)

## Save total Speeds

In [43]:
SHEET_NAME = 'Time Consup. Anlys.S.'

In [44]:
base_path: str = os.getcwd()
path_to_file = os.path.join(base_path, "results")
path_to_file = os.path.join(path_to_file, "redundancy-model-" + MODEL_VERSION_NAME)

count_runs_per_data_set: dict[str, int] = {}
for key in time_recorders.keys():
        idx = 0
        _ =  f"{idx:02d}_{key}.json"
        while os.path.exists(os.path.join(path_to_file, f"{idx:02d}_{key}.json")):
                idx += 1
        count_runs_per_data_set[key] = idx

entries: list[tuple[int,str,str, int, float, float, float]] = []

entry: tuple[int,str,str, int, float, float, float] = None
for key, value in time_recorders.items():
        entry = (key, count_runs_per_data_set[key], MODEL_VERSION_NAME, str(THREADING).lower(), 
                 value.nanoseconds, value.milliseconds, value.seconds, value.minutes)
        entries.append(entry)

columns = ['Dataset', 'Run Count', 'Model Version', 'Threading Enabled', 
           'Nanoseconds', 'Milliseconds', 'Seconds', 'Minutes']

_file_path: str = os.path.join(base_path, os.getenv('OUTPUT_EXCEL_NAME_JUST_TEXTUAL'))
check = os.path.exists(_file_path)

old_time_consuption_data: pd.DataFrame = None
if check:
        try:
                old_time_consuption_data = pd.read_excel(_file_path, SHEET_NAME)
        except ValueError:
                check = False
           
time_consumption_data: pd.DataFrame = pd.DataFrame(entries, columns=columns)

if check and not old_time_consuption_data.empty:
        # Checking if in the excel is already the data. Case: this code is executed twice for the same data
        old_time_consuption_data = old_time_consuption_data.dropna()
        for idx in range(len(old_time_consuption_data)):
                condition = time_consumption_data[
                        (time_consumption_data['Dataset'] == old_time_consuption_data.iat[idx, 0]) & 
                        (time_consumption_data['Run Count'] == old_time_consuption_data.iat[idx, 1])
                ].index
                time_consumption_data = time_consumption_data.drop(condition).reset_index(drop=True)
        time_consumption_data = pd.concat([old_time_consuption_data, time_consumption_data]).reset_index(drop=True)

In [45]:
def formatter_time(wb: Workbook, sheet_name: str): 
    ws = wb[sheet_name]
    header_font = Font(size=14, bold=True)
    for cell in ws["1:1"]:
        cell.font = header_font
    
    ADDITIONAL_LENGTH: int = 0
    ADJUSTED_WIDTH: int = 0
    MAX_LEN: int = 0
    for col in ws.iter_cols(min_row=1, max_row=1):
        for cell in col:
            MAX_LEN = len(str(cell.value))
            ADDITIONAL_LENGTH = (MAX_LEN + 2)
            ADJUSTED_WIDTH = 0
            ADJUSTED_WIDTH =  ADDITIONAL_LENGTH * 1.5
            ws.column_dimensions[utils_get_column_letter(cell.column)].width = ADJUSTED_WIDTH
            
    alignment = Alignment(vertical='center', horizontal='left')
    for row in ws.iter_rows():
        for cell in row:
            cell.alignment = alignment

    num_columns = ws.max_column
    header_range = f"A1:{utils_get_column_letter(num_columns)}1"
    ws.auto_filter.ref = header_range
    ws.freeze_panes = ws['A2']

    wrap_alignment = Alignment(wrap_text=True, vertical='top', horizontal='left')
    for row in ws.iter_rows(min_row=2):
        for cell in row:
            cell.alignment = wrap_alignment

In [46]:
save_to_excel(time_consumption_data, formatter_time, SHEET_NAME, os.getenv("OUTPUT_EXCEL_NAME_JUST_TEXTUAL"))